In [15]:
import keras

from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.models import Model
import numpy as np
import os

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# from AVA_dataloader import train_generator

print(keras.__version__)
os.environ['CUDA_DEVICES_VISIBLE'] = '1'


2.9.0


In [9]:
(images, labels) = train_generator()
print("load  data successfully")

load training data successfully


In [ ]:
# 对图像数据做scale处理
images = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

# 数据集切分
(image_train, image_test, label_train, label_test) = train_test_split(images, labels, test_size=0.25, random_state=42)

# one-hot 编码
label_train = LabelBinarizer().fit_transform(label_train)
label_test = LabelBinarizer().fit_transform(label_test)

# 数据生成器处理
data_generator = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")


In [10]:

# 主程序
def main():
    # 输入图片为256x256，9个分类
    shape, classes = (256, 256, 3), 9
 
    # 调用keras的ResNet50模型
    model = keras.applications.resnet50.ResNet50(input_shape = shape, weights=None, classes=classes)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    # 训练模型
    # training = model.fit(image_train, label_train, epochs=30, batch_size=6)
    training = model.fit_generator(data_generator(image_train, label_train, batch_size = 6),
            validation_data = (image_test, label_test), steps_per_epoch=len(image_train) // 6,
            epochs=30 )

     # 评估模型
    model.evaluate(image_test, label_test, batch_size=32)
 
    # 把训练好的模型保存到文件
    model.save('resnet_ava_classification.h5')
    # model = keras.models.load_model('resnet_model_dog_n_face.h5')
